## Imports

In [6]:
import constants
from constants import *

import os

from matplotlib import pyplot as plt
import pandas as pd

import time

import tensorflow as tf
import numpy as np
import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import *

from sklearn.metrics import *
from sklearn.metrics import classification_report
import seaborn as sns

import sys
import importlib
importlib.reload(constants)

ModuleNotFoundError: No module named 'tensorflow.compat'

## Define Device

In [3]:
from tensorflow.python.client import device_lib

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')


In [5]:
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


## Generación del dataset

In [13]:
train_panda = pd.read_csv(os.path.join(CSV_FOLDER, TRAIN_CSV))
validation_panda = pd.read_csv(os.path.join(CSV_FOLDER, VALIDATION_CSV))

In [14]:
# train_panda["path"] = train_panda["path"].str.replace("animals",DATASETS_ROOT_FOLDER + "/animals")

In [15]:
# Añadir el path absoluto (la parte desde C:// ... hasta animals al inicio para poder acceder
train_panda["path"] = [DATASETS_ROOT_FOLDER + item for item in train_panda["path"]]
validation_panda["path"] = [DATASETS_ROOT_FOLDER + item for item in validation_panda["path"]]

In [16]:
dict_item = {
    "Turtle": 0,
    "Chicken": 1,
    "Bear": 2
}

train_panda["class"] = [dict_item[item] for item in train_panda["class"]]
validation_panda["class"] = [dict_item[item] for item in validation_panda["class"]]

In [17]:
train_panda

,path,class
0,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
1,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
2,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,1
3,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
4,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
...,...,...
7287,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,1
7288,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,1
7289,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
7290,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,1


In [18]:
validation_panda

,path,class
0,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
1,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,2
2,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,2
3,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,2
4,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,2
...,...,...
906,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
907,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,0
908,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,2
909,C:/Users/albam/OneDrive - UNIVERSIDAD DE HUELV...,1


In [19]:
# pixeles a valores entre 0 y 1
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

In [21]:
train_ds = train_datagen.flow_from_dataframe(
                        train_panda,
                        x_col = 'path',
                        y_col = 'class',
                        target_size = (IMG_WIDTH,IMG_HEIGHT),
                        color_mode = 'rgb',
                        batch_size = 32,
                        shuffle = True,
                        interpolation = "bicubic",
                        class_mode='raw'
                        )

# Validation
validation_ds = validation_datagen.flow_from_dataframe(
                        validation_panda,
                        x_col = 'path',
                        y_col = 'class',
                        target_size = (IMG_WIDTH,IMG_HEIGHT),
                        color_mode = 'rgb',
                        batch_size = 32,
                        shuffle = False,    # no necesario para la validacion ya que solo sirve para medir
                        interpolation = "bicubic",
                        class_mode='raw'
                        )

Found 7292 validated image filenames.
Found 911 validated image filenames.


## Modelo

In [22]:
MODEL_NAME = "test_model"
FROM_LOGITS = True

In [23]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3)
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 32)          9248      
                                                                 
 flatten (Flatten)           (None, 288)               0

# Hiperparametros del entramiento

In [25]:
# callback to save the best model

# save best model in validation
# monitoriza el accuracy (mejor de validation)
# solo salva el mejor a true
# no solo salva los pesos, salva modelo + pesos en el mismo file
# en validation se salvará el max
# con freq cada epoch
callback_1 = tf.keras.callbacks.ModelCheckpoint(
os.path.join(MODEL_SAVE_FOLDER,MODEL_NAME), monitor='val_categorical_accuracy', verbose=0, save_best_only=True,
save_weights_only=False, mode='max', save_freq='epoch'
)

# callback to stop the model if it hasnt improved in 50 epochs
callback_2 = tf.keras.callbacks.EarlyStopping(
monitor='val_categorical_accuracy', min_delta=0.001, patience=50, verbose=0, mode='max',
baseline=None, restore_best_weights=True
)


## Define Parameters (Loss, Accuracy and Optimizer)

In [26]:


# A loss object that receives the raw network output and the one-hot raw class labels is created
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=FROM_LOGITS)

# a metric object that calculates the accuracy is created
accuracy = tf.keras.metrics.CategoricalAccuracy()

# optimizer object is created
optimizer = Adam(learning_rate=0.01)

## Compile Model

In [27]:
# compilación del modelo
model.compile(optimizer = optimizer,loss = loss , metrics = accuracy)

# Bucle de entrenamiento

In [ ]:
#   with tf.device('/GPU:0'):
h = model.fit(train_ds, epochs= 2, validation_data = validation_ds, batch_size = 2, callbacks= [callback_1, callback_2])

Epoch 1/2


In [ ]:
# In case it is decided to train more than 100 epochs
for i in range(epochs):

    with tf.device('/device:GPU:0'):
    #SECUENCIA PARA CONFECCIÓN DEL CSV

    history['epoch'] = list(range(0,len(history.index)))

    history.to_csv(os.path.join(CSV_FOLDER, name_csv_history), header=True, index=False)

    # Plot history: CE
    plt.figure()
    plt.plot(history['epoch'], history['loss'], label='Loss (training data)')
    plt.plot(history['epoch'], history['val_loss'], label='Loss (validation data)')
    plt.title('Loss')
    plt.ylabel('CE')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # Plot history: Accuracy
    plt.figure()
    plt.plot(history['epoch'], history['categorical_accuracy'], label='Accuracy (training data)')
    plt.plot(history['epoch'], history['val_categorical_accuracy'], label='Accuracy (validation data)')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # show training time
    fin = time.time()
    print(round((fin - inicio)/60, 0))